In [1]:
# 050224
# testing dataloader

In [2]:
!pip install wandb

In [3]:
# Imports

import cv2
from PIL import Image

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import math as maths

import os
import random

from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
from torch.nn import functional
#from torchsummary import summary
#import torchvision.transforms as transforms

from tqdm import tqdm
from IPython.display import clear_output

import wandb
import pprint


from functions import import_imagedata, get_data, label_oh_tf,  Unwrap, ImageProcessor,IDSWDataSetLoader
from architectures import vgg16net, smallnet1, smallnet2, smallnet3
from loop_fns import loop, test_loop
from fns4wandb import build_optimizer, set_optimizer, hp_sweep, train_model, train_log, log_test_score, set_lossfn, pipeline

from architectures import build_net, smallnet3

wandb.login()

wandb: Currently logged in as: naughticalnonsence (antvis). Use `wandb login --relogin` to force relogin


True

In [4]:
"""from loop_fns import loop

def pipeline(config, col_dict,save_dict, title, device, seed):
    x_train, y_train, x_val, y_val, x_test, y_test = get_data(r'/its/home/nn268/antvis/antvis/optics/AugmentedDS_IDSW/', seed)

    with wandb.init(project=title, config=config):
        config = wandb.config
        model = choose_model(config).to(device) ###
        loss_fn = set_lossfn(config.loss_fn)
        
        #t_save_dict = train_model(model, x_train, y_train, x_val, y_val, loss_fn, config, col_dict, save_dict, device)
        # train model begins
        wandb.watch(model, loss_fn, log='all', log_freq=10)
        sample_count =0
        batch_count = 0
        e_count = 0
        t_loss_list = []
        v_loss_list =[]
        t_predict_list = []
        t_label_list = []
        v_predict_list = []
        v_label_list = []
        t_accuracy_list= []
        v_accuracy_list= []

        optimizer = build_optimizer(model, config.optimizer, config.learning_rate, config.weight_decay)
        scheduler = lr_scheduler.ExponentialLR(optimizer, gamma=config.scheduler, last_epoch=-1)
        
        for epoch in tqdm(range(config.epochs)):            
            t_loss_, predict_list_, t_num_correct, t_label_list_, model, optimizer = loop(model, x_train, y_train, epoch, loss_fn, device, col_dict, config.num_classes, optimizer=optimizer, scheduler=scheduler) #model, x_train, y_train, epoch, loss_fn, device, col_dict, config.num_classes, optimizer=optimizer, scheduler=scheduler
            sample_count += len(x_train)
            t_loss_list.append(t_loss_)
            t_predict_list.append(predict_list_)
            t_label_list.append(t_label_list_)
            t_accuracy_list.append(t_num_correct/len(x_train))
            v_loss_, v_predict_list_, v_num_correct, v_label_list_= loop(model, x_val,y_val, epoch, loss_fn, device,col_dict, config.num_classes, train=False) 
            v_loss_list.append(v_loss_)
            v_predict_list.append(v_predict_list_)
            v_label_list.append(v_label_list_)
            v_accuracy_list.append(v_num_correct/len(x_val))

        save_dict['Current_Epoch'] = config['epochs']
        save_dict['training_samples'] = len(x_train)
        save_dict['validation_samples'] = len(x_val)
        save_dict['t_loss_list'] = t_loss_list #
        save_dict['t_predict_list'] = [[c.to('cpu') for c in k]for k in t_predict_list] #
        save_dict['t_accuracy_list'] = t_accuracy_list #
        save_dict['v_loss_list'] = v_loss_list #
        save_dict['v_predict_list'] = [[c.to('cpu') for c in k]for k in v_predict_list]#
        save_dict['v_accuracy_list'] = v_accuracy_list #
        save_dict['t_labels'] = [[c.to('cpu') for c in k]for k in t_label_list]
        save_dict['v_labels'] = [[c.to('cpu') for c in k] for k in v_label_list]


        test_loop(model, x_test, y_test, loss_fn, device, col_dict, title, config.num_classes) 

        title = save_dict['Run']
        with open(f"/its/home/nn268/antvis/antvis/optics/pickles/{title}.pkl", 'wb+') as f:
            pickle.dump(save_dict, f)

    return model

"""


'from loop_fns import loop\n\ndef pipeline(config, col_dict,save_dict, title, device, seed):\n    x_train, y_train, x_val, y_val, x_test, y_test = get_data(r\'/its/home/nn268/antvis/antvis/optics/AugmentedDS_IDSW/\', seed)\n\n    with wandb.init(project=title, config=config):\n        config = wandb.config\n        model = choose_model(config).to(device) ###\n        loss_fn = set_lossfn(config.loss_fn)\n        \n        #t_save_dict = train_model(model, x_train, y_train, x_val, y_val, loss_fn, config, col_dict, save_dict, device)\n        # train model begins\n        wandb.watch(model, loss_fn, log=\'all\', log_freq=10)\n        sample_count =0\n        batch_count = 0\n        e_count = 0\n        t_loss_list = []\n        v_loss_list =[]\n        t_predict_list = []\n        t_label_list = []\n        v_predict_list = []\n        v_label_list = []\n        t_accuracy_list= []\n        v_accuracy_list= []\n\n        optimizer = build_optimizer(model, config.optimizer, config.learni

In [5]:
""""
grid search
'learning_rate': {
            'values': [6.1e-5, 6.2e-5, 6.3e-5, 6.4e-5]
"""

'"\ngrid search\n\'learning_rate\': {\n            \'values\': [6.1e-5, 6.2e-5, 6.3e-5, 6.4e-5]\n'

In [6]:
config = {
    'method': 'random',
    'metric':{
        'goal': 'minimize',
        'name': 'val_loss'},
    'parameters': {
        'dropout':{
            'values': [0.3, 0.4,0.5]
        },
        'weight_decay':{
            'values': [2e-5,3e-5, 4e-5]
        },
        'epochs':{
            'value': 120
        },
        'lin_layer_size': {
            'values': [100] #, 150, 50
        },
        'first_lin_lay':{
            'values':[67968]#67968 67968
        },
        'optimizer': {
            'values': ['adam']
        },
            'learning_rate': {
                'values': [5e-5, 6e-5, 7e-5]
            },
        'scheduler': {
            'values': [0.2]
        },
        'loss_fn': {
            'values': ['MSE', 'CrossEntropy']
        },
        'data_set':{
            'values':['Augmented']
        },
        'num_classes' : {
            'values':[11]
        },
        'ks': {
            'values': [(3,5)]
        },
        'model_name' : {'values': ['smallnet3']},
        'channels' : {'values': [3]},
        'image_path': {
            'values': [r'/its/home/nn268/antvis/antvis/optics/AugmentedDS_IDSW/']
        }
    }
}




In [7]:
"""
config = dict(
    model_name = 'smallnet3',
    epochs= 120, #120, 
    learning_rate =6.01E-05, #8.22E-05,# 6.40E-05  #8e-5,
    dataset= 'IDSW_Aug',
    architecture ='CNN',
    optimizer= 'adam',
    loss_fn = 'CrossEntropy',
    weight_decay= 3.00E-05, #4e-5, #1.00E-05
    dropout = 0.3, #0.4, #0.5 
    first_lin_lay =1064448, #70272, #67968,#1087488,#1124352,#67968,#1087488, #70272,#1124352, #67968, #1087488, #67968, #1087488,
    lin_layer_size= 100,
    ks =3,#(3,5),
    in_chan = 3,
    num_classes =11,
    scheduler = 0.2,
    channels = 3
)


config = {
     "model_name" : 'smallnet3',
    "epochs" :120, 
    "learning_rate" : 5.97E-05, #6.62E-05, #5.97E-05, #6.01E-05, #6.62E-05, #0.00, 00821591686076769, #8e-5,
    "dataset" : 'IDSW_Aug',
    "architecture" :'CNN',
    "optimizer": 'adam',
    "loss_fn" : 'CrossEntropy',
    "weight_decay": 2e-5, #4e-5, #2e-5, #3.00E-05,
    "dropout" : 0.4, #0.4,
    "first_lin_lay" :267264, #1032192, #4128768, #1032192,#4128768, #267264, #13888, #1055232, #67968,#1055232,
    "lin_layer_size": 100,
    "ks" : [3,5],
    "in_chan" : 3,
    "num_classes" :11,
    "scheduler" : 0.2,
    "channels": 3
     
}
"""



#r'/its/home/nn268/antvis/optics/
#pickles
col_dict = {
    'colour': 'colour',
    'size': [226,72],
    'padding': 5,
    'model_size': '2c2l'
}

title = f"IDSWAug_2c2l_e120_{col_dict['size']}_DL_HPsweep_200224"
save_dict = {'Run' : title,
            'Current_Epoch': 0,
            'save_location' : r'pickles/'}

sweep_id = wandb.sweep(config, project=title+f"_{col_dict['colour']}_{col_dict['size']}")

Create sweep with ID: w7ee0nue
Sweep URL: https://wandb.ai/antvis/IDSWAug_2c2l_e120_%5B226%2C%2072%5D_DL_HPsweep_200224_colour_%5B226%2C%2072%5D/sweeps/w7ee0nue


In [8]:
#from functions import get_data
#from loop_fns import test_loop
"""
def train(config=None):
    # lists for save dict
    t_loss_list = []
    v_loss_list =[]
    t_predict_list = []
    t_label_list = []
    v_predict_list = []
    v_label_list = []
    t_accuracy_list= []
    v_accuracy_list= []
    
    with wandb.init(config=config):
        config = wandb.config
        
        x_train, y_train, x_val, y_val, x_test, y_test = get_data(file_path= r'/its/home/nn268/antvis/antvis/optics/AugmentedDS_IDSW/', seed=seed)
        
        model =smallnet3(in_chan=3, f_lin_lay=67968, l_lin_lay=11, ks=(3,5)).to(device) #10368
        if config.loss_fn == 'MSE':
            loss_fn = nn.MSELoss()
        elif config.loss_fn == 'CrossEntropy':
            loss_fn = nn.CrossEntropyLoss()

        e_count = 0
         # *

        optimizer = build_optimizer(model, config.optimizer, config.learning_rate, config.weight_decay)

        for epoch in range(config.epochs):
            # current_loss, predict_list, num_correct, label_list, model, optimizer
            t_loss, t_predict_list_, t_num_correct, t_label_list_, model, optimizer = loop(model, x_train, y_train, epoch, loss_fn, device, col_dict, num_classes=11, optimizer=optimizer)
            t_accuracy = (t_num_correct /len(x_train))*100
            t_loss_list.append(t_loss)
            t_predict_list.append(t_predict_list_)
            t_label_list.append(t_label_list_)
            t_accuracy_list.append(t_accuracy)

            v_loss, v_predict_list_, v_num_correct, v_label_list_= loop(model, x_val, y_val, epoch, loss_fn, device,col_dict,num_classes=11, train=False)
            v_accuracy= (v_num_correct / len(x_val))*100
            v_loss_list.append(v_loss)
            v_predict_list.append(v_predict_list_)
            v_label_list.append(v_label_list_)
            v_accuracy_list.append(v_accuracy)

            t_avg_loss =t_loss/len(x_train)
            v_avg_loss = v_loss /len(x_val)

            e_count +=1
            # logging
            wandb.log({'avg_train_loss': t_avg_loss, 'epoch':epoch})
            wandb.log({'avg_val_loss': v_avg_loss, 'epoch':epoch})

            wandb.log({'train_loss': t_loss, 'epoch':epoch})
            wandb.log({'val_loss': v_loss, 'epoch':epoch})

            wandb.log({'train_correct': t_num_correct, 'epoch':epoch})
            wandb.log({'val_correct': v_num_correct, 'epoch':epoch})

            wandb.log({'train_accuracy_%': t_accuracy, 'epoch':epoch})
            wandb.log({'val_accuracy_%': v_accuracy, 'epoch':epoch})

            wandb.log({'t_labels': t_label_list, 'epoch':epoch})
            wandb.log({'v_labels': v_label_list, 'epoch':epoch})

            wandb.log({'t_predictions': t_predict_list, 'epoch':epoch})
            wandb.log({'v_predictions': v_predict_list, 'epoch':epoch})

            # add lists to save dict after all epochs run
    save_dict['Current_Epoch'] = config['epochs']
    save_dict['training_samples'] = len(x_train)# should this be the whole list for future graphs...?
    save_dict['validation_samples'] = len(x_val)
    save_dict['t_loss_list'] = t_loss_list #[c.to('cpu') for c in t_loss_list]
    save_dict['t_predict_list'] = [[c.to('cpu') for c in k]for k in t_predict_list] #[[c.to('cpu') for c in k]for k in t_predict_list]  # [c.to('cpu') for c in t_predict_list] 
    save_dict['t_accuracy_list'] = t_accuracy_list #
    save_dict['v_loss_list'] = v_loss_list #[c.to('cpu') for c in v_loss_list]
    save_dict['v_predict_list'] = [[c.to('cpu') for c in k]for k in v_predict_list]#[[c.to('cpu') for c in k]for k in v_predict_list] # [c.to('cpu') for c in v_predict_list]
    save_dict['v_accuracy_list'] = v_accuracy_list #
    save_dict['t_labels'] = [[c.to('cpu') for c in k]for k in t_label_list]
    save_dict['v_labels'] = [[c.to('cpu') for c in k] for k in v_label_list]
        
    test_predictions, test_y, test_accuracy = test_loop(model, x_test, y_test, loss_fn, device, col_dict, title, config.num_classes)
    save_dict['test_predictions']= [c.to('cpu') for c in test_predictions]
    save_dict['test_labels'] = [c.to('cpu') for c in test_y]
    save_dict['test_acc'] = test_accuracy

    title = save_dict['Run']
    with open(f"/its/home/nn268/antvis/antvis/optics/pickles/{title}.pkl", 'wb+') as f:
        pickle.dump(save_dict, f)"""

'\ndef train(config=None):\n    # lists for save dict\n    t_loss_list = []\n    v_loss_list =[]\n    t_predict_list = []\n    t_label_list = []\n    v_predict_list = []\n    v_label_list = []\n    t_accuracy_list= []\n    v_accuracy_list= []\n    \n    with wandb.init(config=config):\n        config = wandb.config\n        \n        x_train, y_train, x_val, y_val, x_test, y_test = get_data(file_path= r\'/its/home/nn268/antvis/antvis/optics/AugmentedDS_IDSW/\', seed=seed)\n        \n        model =smallnet3(in_chan=3, f_lin_lay=67968, l_lin_lay=11, ks=(3,5)).to(device) #10368\n        if config.loss_fn == \'MSE\':\n            loss_fn = nn.MSELoss()\n        elif config.loss_fn == \'CrossEntropy\':\n            loss_fn = nn.CrossEntropyLoss()\n\n        e_count = 0\n         # *\n\n        optimizer = build_optimizer(model, config.optimizer, config.learning_rate, config.weight_decay)\n\n        for epoch in range(config.epochs):\n            # current_loss, predict_list, num_correct, l

In [9]:
print(os.getcwd())

C:\Users\nn268\Desktop


In [10]:
from fns4wandb import hp_sweep_DL
import pickle
import torch.optim.lr_scheduler as lr_scheduler
device = "cuda:1" if torch.cuda.is_available() else "cpu"
from fns4wandb import train

def go(config=None):
    save_dict = {'Run' : title,
            'Current_Epoch': 0,
            'save_location' : r'pickles/'}
    with wandb.init(config=config):
        config = wandb.config
        print(config, type(config))
        model = smallnet3(in_chan=3, f_lin_lay=config.first_lin_lay, l_lin_lay=config.num_classes, ks=config.ks).to(device)


        model, save_dict= hp_sweep_DL(config, col_dict,save_dict,device, seed=4,model=model)

wandb.agent(sweep_id, go, count=5)
"""def tr(config=None, save_dict=save_dict):
    with wandb.init(config=config):
        seed = random.randint(0, 50)
        config = wandb.config
        title = save_dict['Run']
        model = choose_model(config).to(device)
        loss_fn = set_lossfn(config.loss_fn)
        optimizer = build_optimizer(model, config.optimizer, config.learning_rate, config.weight_decay)
        scheduler = lr_scheduler.ExponentialLR(optimizer, gamma=config.scheduler, last_epoch=-1)

        model, save_dict,x_test, y_test = hp_sweep(config, col_dict, save_dict, device, seed, model, loss_fn, optimizer, scheduler)
        test_predict_list, test_labels, test_accuracy = test_loop(model, x_test, y_test, loss_fn, device, col_dict, title, config.num_classes)
        save_dict['test_predict_list'] = [c.to('cpu') for c in test_predict_list]
        save_dict['test_labels'] = test_labels#[c.to('cpu') for c in test_labels]
        save_dict['test_accuracy'] = test_accuracy
        save_dict['seed'] = seed
        
        with open(f"/its/home/nn268/antvis/antvis/optics/pickles/{title}.pkl", 'wb+') as f:
            pickle.dump(save_dict, f)"""

    #wandb.agent(sweep_id, tr, count=25)
    # config, col_dict,save_dict, title, device, seed)


    #from fns4wandb import train_DL
    #model = pipeline(config, col_dict,save_dict, title="2c2l_training_113x36, 3chan", device=device, seed=seed)
    #model = train(device,col_dict, save_dict, config)
    #model = train_DL(device,col_dict, save_dict, config)
    #pipeline(config, col_dict, save_dict, title, device=device)



wandb: Agent Starting Run: ejuxjj0s with config:
wandb: 	channels: 3
wandb: 	data_set: Augmented
wandb: 	dropout: 0.5
wandb: 	epochs: 120
wandb: 	first_lin_lay: 67968
wandb: 	image_path: /its/home/nn268/antvis/antvis/optics/AugmentedDS_IDSW/
wandb: 	ks: [3, 5]
wandb: 	learning_rate: 5e-05
wandb: 	lin_layer_size: 100
wandb: 	loss_fn: CrossEntropy
wandb: 	model_name: smallnet3
wandb: 	num_classes: 11
wandb: 	optimizer: adam
wandb: 	scheduler: 0.2
wandb: 	weight_decay: 2e-05


{'channels': 3, 'data_set': 'Augmented', 'dropout': 0.5, 'epochs': 120, 'first_lin_lay': 67968, 'image_path': '/its/home/nn268/antvis/antvis/optics/AugmentedDS_IDSW/', 'ks': [3, 5], 'learning_rate': 5e-05, 'lin_layer_size': 100, 'loss_fn': 'CrossEntropy', 'model_name': 'smallnet3', 'num_classes': 11, 'optimizer': 'adam', 'scheduler': 0.2, 'weight_decay': 2e-05} <class 'wandb.sdk.wandb_config.Config'>


  0%|          | 0/120 [00:00<?, ?it/s]
Traceback (most recent call last):
  File "C:\Users\nn268\AppData\Local\Temp\ipykernel_10960\1867843242.py", line 17, in go
    model, save_dict= hp_sweep_DL(config, col_dict,save_dict,device, seed=4,model=model)
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\nn268\Desktop\fns4wandb.py", line 266, in hp_sweep_DL
    t_loss, t_predictions, t_num_correct,t_labels, model, optimizer = batch_loop(model, train_loader, epoch, loss_fn, device, col_dict, num_classes=config.num_classes, pad_size=col_dict['padding'],optimizer=optimizer, scheduler=scheduler)
                                                                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\nn268\Desktop\loop_fns.py", line 219, in batch_loop
    #for idx, img in enumerate(X):
  Fil

training...


Run ejuxjj0s errored:
Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\Lib\site-packages\wandb\agents\pyagent.py", line 308, in _run_job
    self._function()
  File "C:\Users\nn268\AppData\Local\Temp\ipykernel_10960\1867843242.py", line 17, in go
    model, save_dict= hp_sweep_DL(config, col_dict,save_dict,device, seed=4,model=model)
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\nn268\Desktop\fns4wandb.py", line 266, in hp_sweep_DL
    t_loss, t_predictions, t_num_correct,t_labels, model, optimizer = batch_loop(model, train_loader, epoch, loss_fn, device, col_dict, num_classes=config.num_classes, pad_size=col_dict['padding'],optimizer=optimizer, scheduler=scheduler)
                                                                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File 

...

{'channels': 3, 'data_set': 'Augmented', 'dropout': 0.4, 'epochs': 120, 'first_lin_lay': 67968, 'image_path': '/its/home/nn268/antvis/antvis/optics/AugmentedDS_IDSW/', 'ks': [3, 5], 'learning_rate': 7e-05, 'lin_layer_size': 100, 'loss_fn': 'CrossEntropy', 'model_name': 'smallnet3', 'num_classes': 11, 'optimizer': 'adam', 'scheduler': 0.2, 'weight_decay': 4e-05} <class 'wandb.sdk.wandb_config.Config'>


  0%|          | 0/120 [00:00<?, ?it/s]
Traceback (most recent call last):
  File "C:\Users\nn268\AppData\Local\Temp\ipykernel_10960\1867843242.py", line 17, in go
    model, save_dict= hp_sweep_DL(config, col_dict,save_dict,device, seed=4,model=model)
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\nn268\Desktop\fns4wandb.py", line 266, in hp_sweep_DL
    t_loss, t_predictions, t_num_correct,t_labels, model, optimizer = batch_loop(model, train_loader, epoch, loss_fn, device, col_dict, num_classes=config.num_classes, pad_size=col_dict['padding'],optimizer=optimizer, scheduler=scheduler)
                                                                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\nn268\Desktop\loop_fns.py", line 219, in batch_loop
    #for idx, img in enumerate(X):
  Fil

training...


Run 37kbod8l errored:
Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\Lib\site-packages\wandb\agents\pyagent.py", line 308, in _run_job
    self._function()
  File "C:\Users\nn268\AppData\Local\Temp\ipykernel_10960\1867843242.py", line 17, in go
    model, save_dict= hp_sweep_DL(config, col_dict,save_dict,device, seed=4,model=model)
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\nn268\Desktop\fns4wandb.py", line 266, in hp_sweep_DL
    t_loss, t_predictions, t_num_correct,t_labels, model, optimizer = batch_loop(model, train_loader, epoch, loss_fn, device, col_dict, num_classes=config.num_classes, pad_size=col_dict['padding'],optimizer=optimizer, scheduler=scheduler)
                                                                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File 

'def tr(config=None, save_dict=save_dict):\n    with wandb.init(config=config):\n        seed = random.randint(0, 50)\n        config = wandb.config\n        title = save_dict[\'Run\']\n        model = choose_model(config).to(device)\n        loss_fn = set_lossfn(config.loss_fn)\n        optimizer = build_optimizer(model, config.optimizer, config.learning_rate, config.weight_decay)\n        scheduler = lr_scheduler.ExponentialLR(optimizer, gamma=config.scheduler, last_epoch=-1)\n\n        model, save_dict,x_test, y_test = hp_sweep(config, col_dict, save_dict, device, seed, model, loss_fn, optimizer, scheduler)\n        test_predict_list, test_labels, test_accuracy = test_loop(model, x_test, y_test, loss_fn, device, col_dict, title, config.num_classes)\n        save_dict[\'test_predict_list\'] = [c.to(\'cpu\') for c in test_predict_list]\n        save_dict[\'test_labels\'] = test_labels#[c.to(\'cpu\') for c in test_labels]\n        save_dict[\'test_accuracy\'] = test_accuracy\n        s

In [11]:
# /smbhome.uscs.susx.ac.uk
import os
if os.path.exists(r'//smbhome.uscs.susx.ac.uk/nn268/Documents/PHD/antvis/optics/AugmentedDS_IDSW'):
    print('True')
else:
    print('false')

True


In [12]:
n = '/its/home/nn268/antvis/antvis/optics/AugmentedDS_IDSW/IDSW003_060423_1133_SW_0029.JPG_Augmented_left_1.JPG'
print(len(n))
print(n[59:61])

106
03


In [13]:
# 8.640776699029127


In [14]:
"""def tr(config=None):
    with wandb.init(config=config):
        config = wandb.config
        device = "cuda:0" if torch.cuda.is_available() else "cpu"
        model = hp_sweep(config, col_dict, save_dict, device)"""

'def tr(config=None):\n    with wandb.init(config=config):\n        config = wandb.config\n        device = "cuda:0" if torch.cuda.is_available() else "cpu"\n        model = hp_sweep(config, col_dict, save_dict, device)'

In [15]:
#wandb.agent(sweep_id, tr, count=25)

In [16]:
import os
print(os.getcwd())

C:\Users\nn268\Desktop


In [17]:
#model = pipeline(config, col_dict, title="2c2l_training_113x36, 3chan", image_file_path= "/its/home/nn268/optics/AugmentedDS_IDSW/")